In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def Rastrigin(X, A=10):
    x = np.asarray(X)
    n = len(x)
    return A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))

Rastrigin_limits = [[-5.12, 5.12]]

def Rozenbrock1(X):
    x, y, = X
    f1 = (x-1)**3 - y + 1 < 0
    f2 = x + y - 2 < 0
    if all([f1, f2]):
        return (1 - x)**2 + 100*(y - x**2)**2
    else:
        return np.finfo(float).max

Rozenbrock1_limits = [[-1.5, 1.5], [-0.5, 2.5]]

def Rozenbrock2(X):
    x, y, = X
    f1 = x**2+y**2 < 2
    if f1:
        return (1 - x)**2 + 100*(y - x**2)**2
    else:
        return np.finfo(float).max

Rozenbrock2_limits = [[-1.5, 1.5], [-1.5, 1.5]]

def Mishri_Berda(X):
    x, y = X
    f1 = (x+5)**2+(y+5)**2 < 25
    if f1:
        return np.exp((1-np.cos(x))**2)*np.sin(y) + np.exp((1-np.sin(y))**2)*np.cos(x) + (x-y)**2
    else:
        return np.finfo(float).max

Mishri_Berda_limits = [[-10, 0], [-6.5, 0]]

def Siminonesku(X):
    x, y, = X
    f1 = x**2+y**2 < (1 + 0.2*np.cos(8*np.arctan(x/y)))**2
    if f1:
        return 0.1*x*y
    else:
        return np.finfo(float).max


Siminonesku_limits = [[-1.25, 1.25], [-1.25, 1.25]]

def Reductor(X):
    x1, x2, x3, x4, x5, x6, x7, = X
    x3 = int(np.clip(round(x3), 17, 28))
    
    f1 = 27 / (x1 * x2**2 * x3) - 1  <= 0
    f2 = 397.5 / (x1 * x2**2 * x3**2) - 1 <= 0
    f3 = 1.93 * x4**3 / (x2 * x3 * x6**4) - 1 <= 0
    f4 = 1.93 / (x2 * x3 * x7**4) - 1 <= 0
    f5 = 1.0/(110 * x6**3) * np.sqrt(((745*x4) / (x2 * x3))**2 + 16.9 * 10**6) - 1 <= 0
    f6 = 1.0/(85 * x7**3) * np.sqrt(((745*x5) / (x2 * x3))**2 + 157.5 * 10**6) - 1 <= 0
    f7 = (x2*x3) / 40 - 1 <= 0
    f8 = 5*x2 / x1 - 1 <= 0
    f9 = x1 / (12 * x2) - 1 <= 0
    f10 = (1.5 * x6 + 1.9) / x4 - 1 <= 0
    f11 = (1.1 * x7 + 1.9) / x5 - 1 <= 0
    if all([f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11]):
        return 0.7854*x1*x2**2*(3.3333*x3**2 + 14.9334*x3 - 43.0934) - 1.508*x1*(x6**2 + x7**2) + 7.4777*(x6**3 + x7**3) + 0.7854*(x4*x6**2 + x5*x7**2)
    return np.finfo(float).max

Reductor_limits = [[2.6, 3.6], [0.7, 0.8], [17, 28], [7.3, 8.3], [7.8, 8.3], [2.9, 3.9], [5.0, 5.5]]

def Trail(X):
    x1, x2, x3 = X
    denominator = x1**2 + x2**2

    penalties = 0.0

    f1 = 1 - (x2**3 * x3) / (7.178 * x1**4)
    f2 = (4*x2**2 - x1*x2) / denominator + 1/(5.108*x1**2) - 1
    f3 = 1 - (140.45*x1) / (x2**2 * x3)
    f4 = (x2 + x1) / 1.5 - 1
    
    if f1 < 0:
        penalties += abs(f1)
    if f2 > 0:
        penalties += abs(f2)
    if f3 < 0:
        penalties += abs(f3)
    if f4 > 0:
        penalties += abs(f4)

    base_cost = (x3 + 2) * x2 * x1**2
    return base_cost + 1e6 * penalties  # великий штраф


Trail_limits = [[0.005, 2.0], [0.25, 1.3], [2.0, 15.0]]

In [35]:
class Particle:
    def __init__(self, dim, pos_bounds, vel_bounds):
        """
        Initializes a particle with random position and velocity.
        
        Parameters:
            dim (int): Dimensionality of the search space.
            pos_bounds (tuple): (x_min, x_max) boundaries for positions.
            vel_bounds (tuple): (v_min, v_max) boundaries for velocities.
        """
        self.dim = dim
        self.pos_bounds = np.tile(np.array(pos_bounds), (dim, 1)) if np.array(pos_bounds).ndim == 1 else np.array(pos_bounds)
        self.vel_bounds = np.tile(np.array(vel_bounds), (dim, 1)) if np.array(vel_bounds).ndim == 1 else np.array(vel_bounds)
        self.position = np.random.uniform(self.pos_bounds[:, 0], self.pos_bounds[:, 1])
        self.velocity = np.random.uniform(self.vel_bounds[:, 0], self.vel_bounds[:, 1])
        self.best_position = self.position.copy()
        self.best_score = float('inf')

    

    def update_velocity(self, global_best, w, alpha1, alpha2):
        """
        Updates the particle's velocity based on inertia, cognitive, and social components.
        
        Parameters:
            global_best (ndarray): The best position found by the swarm.
            w (float): Inertia weight.
            alpha1 (float): Cognitive acceleration coefficient.
            alpha2 (float): Social acceleration coefficient.
        """
        r1, r2 = np.random.rand(2)
        cognitive = alpha1 * r1 * (self.best_position - self.position)
        social = alpha2 * r2 * (global_best - self.position)
        self.velocity = w * self.velocity + cognitive + social
        self.velocity = np.clip(self.velocity, self.vel_bounds[:, 0], self.vel_bounds[:, 1])


    def update_position(self):
        """
        Updates the particle's position based on its velocity and applies boundary reflection.
        """
        self.position += self.velocity
        # Reflect the particle if it goes out of bounds
        for j in range(self.dim):
            if self.position[j] < self.pos_bounds[j, 0]:
                self.position[j] = self.pos_bounds[j, 0] + abs(self.position[j] - self.pos_bounds[j, 0])
                self.velocity[j] = -self.velocity[j]
            elif self.position[j] > self.pos_bounds[j, 1]:
                self.position[j] = self.pos_bounds[j, 1] - abs(self.position[j] - self.pos_bounds[j, 1])
                self.velocity[j] = -self.velocity[j]

class PSO:
    def __init__(self, fitness_func, num_particles, dim, pos_bounds, vel_bounds, max_iter, 
                 alpha1, alpha2, w_max, w_min, 
                 enable_shaking=False, stagnation_iter=20, shake_probability=0.5, shake_amplitude=0.1,
                 target_global_min=None, epsilon_target=None, epsilon_stagnation=1e-6,
                 swarm_with_limitation=False):
        self.fitness_func = fitness_func
        self.num_particles = num_particles
        self.dim = dim
        self.pos_bounds = pos_bounds
        if np.array(vel_bounds).ndim == 1:
            self.vel_bounds = np.tile(np.array(vel_bounds), (dim, 1))
        else:
            self.vel_bounds = np.array(vel_bounds)
        self.max_iter = max_iter
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.w_max = w_max
        self.w_min = w_min

        self.target_global_min = target_global_min
        self.epsilon_target = epsilon_target
        self.stagnation_iter = stagnation_iter

        self.enable_shaking = enable_shaking
        self.epsilon_stagnation = epsilon_stagnation
        self.shake_probability = shake_probability
        self.shake_amplitude = shake_amplitude
        if swarm_with_limitation:
            self._create_swarm()
        else:
            self.swarm = [Particle(dim, pos_bounds, vel_bounds) for _ in range(num_particles)]
        self.global_best_position = None
        self.global_best_score = float('inf')

        self.history_best = []
        self.history_all_positions = []

        self._initialize_global_best()
    
    def _create_valid_particle(self):
        """
        Створює одну валідну частинку (позиція + швидкість)
        """
        pos_bounds = np.array(self.pos_bounds)
        vel_bounds = np.array(self.vel_bounds)

        for _ in range(100):
            position = np.random.uniform(pos_bounds[:,0], pos_bounds[:,1])
            velocity = np.random.uniform(vel_bounds[:,0], vel_bounds[:,1])
            score = self.fitness_func(position)
            if np.isfinite(score) and score < 1e6:
                return position, velocity

        # fallback — якщо не знайдено валідну
        position = np.random.uniform(pos_bounds[:,0], pos_bounds[:,1])
        velocity = np.random.uniform(vel_bounds[:,0], vel_bounds[:,1])
        return position, velocity
    
    def _create_swarm(self):
        """
        Створення всього початкового рою.
        """
        self.swarm = []
        for _ in range(self.num_particles):
            position, velocity = self._create_valid_particle()
            particle = Particle(self.dim, self.pos_bounds, self.vel_bounds)
            particle.position = position
            particle.velocity = velocity
            particle.best_position = position.copy()
            particle.best_score = self.fitness_func(position)
            self.swarm.append(particle)

    def _initialize_global_best(self):
        for particle in self.swarm:
            score = self.fitness_func(particle.position)
            particle.best_score = score
            if np.isfinite(score) and score < self.global_best_score:
                self.global_best_score = score
                self.global_best_position = particle.position.copy()

        if self.global_best_position is None:
            raise ValueError("No valid initial particles found. Try increasing number of particles or adjusting position bounds.")

    def _update_particles(self, w):
        for particle in self.swarm:
            particle.update_velocity(self.global_best_position, w, self.alpha1, self.alpha2)
            particle.update_position()
            score = self.fitness_func(particle.position)
            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = particle.position.copy()
            if score < self.global_best_score:
                self.global_best_score = score
                self.global_best_position = particle.position.copy()

    def _record_history(self):
        """
        Saves the global best position and its score for the current iteration,
        and all particles' positions.
        """
        self.history_best.append((self.global_best_position.copy(), self.global_best_score))

        iteration_positions = [p.position.copy() for p in self.swarm]
        self.history_all_positions.append(iteration_positions)


    def _check_stagnation(self, previous_best, stagnation_count):
        if abs(previous_best - self.global_best_score) < self.epsilon_stagnation:
            return stagnation_count + 1, previous_best
        return 0, self.global_best_score

    def _check_termination(self, iteration):
        if self.target_global_min is not None:
            if self.global_best_score <= self.target_global_min:
                print(f"Target global minimum {self.target_global_min} reached at iteration {iteration}. Stopping.")
                return True
            if self.epsilon_target is not None and abs(self.global_best_score - self.target_global_min) < self.epsilon_target:
                print(f"Global minimum is within epsilon {self.epsilon_target} of target at iteration {iteration}. Stopping.")
                return True
        return False

    def shake_particles(self):
        range_pos = np.abs(np.max(self.pos_bounds) - np.min(self.pos_bounds))
        shake_std = np.maximum(self.shake_amplitude * range_pos, 1e-6)

        shaken_count = 0
        for particle in self.swarm:
            if np.random.rand() < self.shake_probability:
                noise = np.random.normal(0, shake_std, self.dim)
                particle.position += noise
                particle.velocity = np.random.uniform(self.vel_bounds[0], self.vel_bounds[1], self.dim)
                particle.best_position = particle.position.copy()
                particle.best_score = self.fitness_func(particle.position)
                shaken_count += 1
        print(f"Shaken {shaken_count} particles.")

    def run(self):
        stagnation_count = 0
        previous_best = self.global_best_score

        for iteration in range(self.max_iter):
            w = self.w_max - ((self.w_max - self.w_min) * iteration / self.max_iter)

            self._update_particles(w)
            self._record_history()
            stagnation_count, previous_best = self._check_stagnation(previous_best, stagnation_count)

            if self.enable_shaking and stagnation_count >= self.stagnation_iter:
                print(f"Stagnation reached at iteration {iteration}. Shaking particles...")
                self.shake_particles()
                stagnation_count = 0

            if self._check_termination(iteration):
                break

            if iteration % 10 == 0:
                print(f"Iteration {iteration}: Global best score = {self.global_best_score}")

        return self.global_best_position, self.global_best_score, self.history_best, self.history_all_positions


In [4]:
class Bee:
    def __init__(self, dim, pos_bounds):
        self.dim = dim
        self.pos_bounds = np.array(pos_bounds)
        self.position = np.random.uniform(self.pos_bounds[:, 0], self.pos_bounds[:, 1])
        self.score = float('inf')

class BeesAlgorithm:
    def __init__(self, fitness_func, num_bees, dim, pos_bounds, max_iter, elite_sites, best_sites, elite_bees, recruited_bees, patch_shrink=0.95):
        self.fitness_func = fitness_func
        self.num_bees = num_bees
        self.dim = dim
        self.pos_bounds =  np.array(pos_bounds) 
        self.max_iter = max_iter

        self.elite_sites = elite_sites
        self.best_sites = best_sites
        self.elite_bees = elite_bees
        self.recruited_bees = recruited_bees
        self.patch_size = np.abs(np.array(pos_bounds)[:,1] - np.array(pos_bounds)[:,0]) * 0.1
        self.patch_shrink = patch_shrink

        self.bees = [Bee(dim, pos_bounds) for _ in range(num_bees)]
        self.best_position = None
        self.best_score = float('inf')

        self.history_best = []
        self.history_all_positions = []

    def _neighborhood_search(self, site_position, num_bees):
        candidates = []
        for _ in range(num_bees):
            offset = np.random.uniform(-self.patch_size, self.patch_size)
            candidate = np.clip(site_position + offset, self.pos_bounds[:, 0], self.pos_bounds[:, 1])
            candidates.append(candidate)
        return candidates

    def run(self):
        for iteration in range(self.max_iter):
            for bee in self.bees:
                bee.score = self.fitness_func(bee.position)

            self.bees.sort(key=lambda b: b.score)
            new_positions = []

            for i, site in enumerate(self.bees[:self.best_sites]):
                n_recruited = self.elite_bees if i < self.elite_sites else self.recruited_bees
                candidates = self._neighborhood_search(site.position, n_recruited)
                best_candidate = min(candidates, key=lambda pos: self.fitness_func(pos))
                new_positions.append(best_candidate)

            for _ in range(self.num_bees - len(new_positions)):
                random_position = np.random.uniform(self.pos_bounds[:,0], self.pos_bounds[:,1])
                new_positions.append(random_position)

            for bee, pos in zip(self.bees, new_positions):
                bee.position = pos

            best_bee = min(self.bees, key=lambda b: b.score)
            if best_bee.score < self.best_score:
                self.best_score = best_bee.score
                self.best_position = best_bee.position.copy()

            self.patch_size *= self.patch_shrink

            self.history_best.append((self.best_position.copy(), self.best_score))
            self.history_all_positions.append([b.position.copy() for b in self.bees])

        return self.best_position, self.best_score, self.history_best, self.history_all_positions


In [5]:
class Firefly:
    def __init__(self, dim, pos_bounds):
        self.dim = dim
        self.pos_bounds = np.array(pos_bounds)
        self.position = np.random.uniform(self.pos_bounds[:, 0], self.pos_bounds[:, 1])
        self.score = float('inf')

class FireflyAlgorithm:
    def __init__(self, fitness_func, num_fireflies, dim, pos_bounds, max_iter, alpha=0.2, beta0=1.0, gamma=1.0):
        self.fitness_func = fitness_func
        self.num_fireflies = num_fireflies
        self.dim = dim
        self.pos_bounds =  np.array(pos_bounds) 
        self.max_iter = max_iter

        self.alpha = alpha
        self.beta0 = beta0
        self.gamma = gamma

        self.fireflies = [Firefly(dim, pos_bounds) for _ in range(num_fireflies)]
        self.best_position = None
        self.best_score = float('inf')

        self.history_best = []
        self.history_all_positions = []

    def _attractiveness(self, distance):
        return self.beta0 * np.exp(-self.gamma * distance**2)

    def run(self):
        for iteration in range(self.max_iter):
            for firefly in self.fireflies:
                firefly.score = self.fitness_func(firefly.position)

            for i in range(self.num_fireflies):
                for j in range(self.num_fireflies):
                    if self.fireflies[j].score < self.fireflies[i].score:
                        distance = np.linalg.norm(self.fireflies[i].position - self.fireflies[j].position)
                        beta = self._attractiveness(distance)
                        rand = np.random.uniform(-0.5, 0.5, self.dim)
                        self.fireflies[i].position += beta * (self.fireflies[j].position - self.fireflies[i].position) + self.alpha * rand
                        self.fireflies[i].position = np.clip(self.fireflies[i].position, self.pos_bounds[:, 0], self.pos_bounds[:, 1])

            best_firefly = min(self.fireflies, key=lambda f: f.score)
            if best_firefly.score < self.best_score:
                self.best_score = best_firefly.score
                self.best_position = best_firefly.position.copy()

            self.history_best.append((self.best_position.copy(), self.best_score))
            self.history_all_positions.append([f.position.copy() for f in self.fireflies])

        return self.best_position, self.best_score, self.history_best, self.history_all_positions

In [6]:
dim = 10
pso = PSO(
    fitness_func=Rastrigin, 
    num_particles=400, 
    dim=dim, 
    pos_bounds = Rastrigin_limits*dim, 
    vel_bounds = [-1, 1],
    max_iter=1000,
    alpha1=2.0, 
    alpha2=1.0,
    w_max=0.9, 
    w_min=0.5,
    enable_shaking=False,
    stagnation_iter=20,
    shake_probability=0.9,
    shake_amplitude=0.9
)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 83.48049288732315
Iteration 10: Global best score = 29.220315560632784
Iteration 20: Global best score = 11.263763070360994
Iteration 30: Global best score = 8.06918503683795
Iteration 40: Global best score = 6.718562708881407
Iteration 50: Global best score = 6.039242501936883
Iteration 60: Global best score = 6.0083887203627455
Iteration 70: Global best score = 5.1797094420710295
Iteration 80: Global best score = 4.426963120570733
Iteration 90: Global best score = 4.253622270045227
Iteration 100: Global best score = 4.093354770962748
Iteration 110: Global best score = 4.008218044415273
Iteration 120: Global best score = 3.1574420846322226
Iteration 130: Global best score = 3.102913121824386
Iteration 140: Global best score = 3.099248876574862
Iteration 150: Global best score = 3.0321347534161305
Iteration 160: Global best score = 2.0347047850263493
Iteration 170: Global best score = 2.0347047850263493
Iteration 180: Global best score = 2.0201783978388

In [7]:
dim = 2
pso = PSO(
    fitness_func=Rozenbrock2, 
    num_particles=100, 
    dim=dim, 
    pos_bounds=Rozenbrock1_limits, 
    vel_bounds=[-1, 1],
    max_iter=300,
    alpha1=2.0, 
    alpha2=1.0,
    w_max=0.9, 
    w_min=0.4,
    enable_shaking=True,
    stagnation_iter=10,
    shake_probability=0.7,
    shake_amplitude=0.9
)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 0.33167568573367745
Iteration 10: Global best score = 0.012143247141415549
Iteration 20: Global best score = 0.000795293708403443
Iteration 30: Global best score = 0.0005627204833378273
Stagnation reached at iteration 37. Shaking particles...
Shaken 66 particles.
Iteration 40: Global best score = 0.0005627204833378273
Iteration 50: Global best score = 0.00039806432168467266
Stagnation reached at iteration 54. Shaking particles...
Shaken 67 particles.
Iteration 60: Global best score = 0.00039806432168467266
Stagnation reached at iteration 64. Shaking particles...
Shaken 66 particles.
Iteration 70: Global best score = 0.0001294646323787329
Stagnation reached at iteration 75. Shaking particles...
Shaken 65 particles.
Iteration 80: Global best score = 0.0001294646323787329
Stagnation reached at iteration 85. Shaking particles...
Shaken 68 particles.
Iteration 90: Global best score = 0.0001294646323787329
Stagnation reached at iteration 95. Shaking particles

In [23]:
dim = 7
pso = PSO(
    fitness_func=Reductor, 
    num_particles=300, 
    dim=dim, 
    pos_bounds=Reductor_limits, 
    vel_bounds=[-1, 1],
    max_iter=500,
    alpha1=2.0, 
    alpha2=1.0,
    w_max=0.9, 
    w_min=0.4,
)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 3541.1420413090086
Iteration 10: Global best score = 3541.1420413090086
Iteration 20: Global best score = 3498.629849710673
Iteration 30: Global best score = 3263.4160181085977
Iteration 40: Global best score = 3263.4160181085977
Iteration 50: Global best score = 3263.4160181085977
Iteration 60: Global best score = 3184.120616455732
Iteration 70: Global best score = 3050.348224552568
Iteration 80: Global best score = 3041.9701621796944
Iteration 90: Global best score = 3022.8724512929216
Iteration 100: Global best score = 3010.519679454476
Iteration 110: Global best score = 3008.173076322742
Iteration 120: Global best score = 3006.167397906577
Iteration 130: Global best score = 3002.94545292996
Iteration 140: Global best score = 2999.543998197367
Iteration 150: Global best score = 2997.6199245614234
Iteration 160: Global best score = 2997.578700362692
Iteration 170: Global best score = 2997.475987875907
Iteration 180: Global best score = 2996.5587004932

In [37]:
dim = 3
pso = PSO(
    fitness_func=Trail, 
    num_particles=1000, 
    dim=dim, 
    pos_bounds=Trail_limits, 
    vel_bounds=[-1, 1],
    max_iter=500,
    alpha1=2.0, 
    alpha2=2.0,
    w_max=0.9, 
    w_min=0.4,
    swarm_with_limitation=True

)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 8545601.243509017
Iteration 10: Global best score = 8170319.881922522
Iteration 20: Global best score = 8156930.750252747
Iteration 30: Global best score = 8155141.613132592
Iteration 40: Global best score = 8155141.613132592
Iteration 50: Global best score = 8155141.613132592
Iteration 60: Global best score = 8154370.685960778
Iteration 70: Global best score = 8154298.624651783
Iteration 80: Global best score = 8154298.624651783
Iteration 90: Global best score = 8154177.3327093
Iteration 100: Global best score = 8154147.975120243
Iteration 110: Global best score = 8154142.392808328
Iteration 120: Global best score = 8154142.392808328
Iteration 130: Global best score = 8154135.980439524
Iteration 140: Global best score = 8154129.701402739
Iteration 150: Global best score = 8154129.701402739
Iteration 160: Global best score = 8154128.552652151
Iteration 170: Global best score = 8154128.122163961
Iteration 180: Global best score = 8154127.864072869
Iterat

In [21]:
dim = 7
bees = BeesAlgorithm(
    fitness_func=Reductor,
    num_bees=600,
    dim=dim,
    pos_bounds=Reductor_limits,
    max_iter=300,
    elite_sites=10,
    best_sites=20,
    elite_bees=14,
    recruited_bees=7
)

best_pos, best_score, history_best, history_all_positions = bees.run()

print(f"Best position: {best_pos}")
print(f"Best score: {best_score}")

Best position: [ 3.5         0.7        17.          7.37105591  8.05701691  3.35034809
  5.28677245]
Best score: 3002.707354795571


In [13]:
dim = 2
firefly = FireflyAlgorithm(
    fitness_func=Rastrigin,
    num_fireflies=100,
    dim=dim,
    pos_bounds=Rastrigin_limits * dim,
    max_iter=300,
    alpha=0.25,
    beta0=0.5,
    gamma=0.2
)

best_pos, best_score, history_best, history_all_positions = firefly.run()

print(f"Best position: {best_pos}")
print(f"Best score: {best_score}")

Best position: [ 2.57182303e-04 -7.93542925e-05]
Best score: 1.4371488383346787e-05
